In [1]:
import pandas as pd
import psycopg2

#establishing the connection
conn = psycopg2.connect(
    database="loftsvc", 
    user='USER', # Enter your AWS username here
    password='PASSWORD', # Enter your database password here
    host='HOST', # Enter your AWS database server endpoint here
    port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

query = '''CREATE TABLE IF NOT EXISTS Building (
           bldgid INT PRIMARY KEY,
           bldgname VARCHAR(100) NOT NULL,
           unitrange VARCHAR(30)
        );'''

cursor.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS ExceptionLog (
           exceptid SERIAL PRIMARY KEY,
           record TEXT,
           fixstamp TIMESTAMP
        );'''

cursor.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS ReqType (
           rtid INT PRIMARY KEY,
           typename VARCHAR(30) NOT NULL,
           typedesc VARCHAR(255)
        );'''

cursor.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS ServReq (
           srid SERIAL PRIMARY KEY,
           tenantid INT NOT NULL,
           rtid INT NOT NULL,
           bldgid INT NOT NULL,
           reqdate DATE NOT NULL,
           unit SMALLINT NOT NULL,
           reqdescription TEXT,
           compstamp TIMESTAMP
        );'''

cursor.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS Tenant (
           tenantid SERIAL PRIMARY KEY,
           fname VARCHAR(50) NOT NULL,
           lname VARCHAR(50) NOT NULL,
           phonenum CHAR(14) NOT NULL
        );'''

cursor.execute(query)
conn.commit()

In [2]:
### DO NOT RUN THIS CELL MULTIPLE TIMES WITHOUT DELETING THE DATA OUT OF THE TABLE FIRST ###
### Creates the Building table -- should have 8 records when complete ###

df = pd.read_csv('building.csv')

for x in df.index:
    query = f'''INSERT INTO Building (bldgid, bldgname, unitrange)
            VALUES ({df['bldgid'].loc[x]}, '{df['bldgname'].loc[x]}', '{df['unitrange'].loc[x]}')'''
    
    cursor.execute(query)
    
    conn.commit()
    
query = '''SELECT * FROM Building;'''
df = pd.read_sql(query, conn, index_col='bldgid')
print(df)

              bldgname unitrange
bldgid                          
1             Antietam   101-198
2             Bull Run   201-298
3           Charleston   301-398
4              Detroit   401-498
5         Elkins Ferry   501-598
6       Fredericksburg   601-698
7           Gettysburg   701-798
8               Hudson   801-898


/var/folders/44/s9hsvm1943z9xb042mqxxtgc0000gn/T/ipykernel_34180/4127244328.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, index_col='bldgid')


In [3]:
### DO NOT RUN THIS CELL MULTIPLE TIMES WITHOUT DELETING THE DATA OUT OF THE TABLE FIRST ###
### Creates the ExceptionLog table -- should have 20 records when complete ###

df = pd.read_csv('except.csv')

for x in df.index:
    
    df.replace("\'","",inplace=True)
    
    query = f"""INSERT INTO ExceptionLog (record)
            VALUES ('{df['record'].loc[x]}')"""
    
    cursor.execute(query)
    
    conn.commit()
    
query = '''SELECT COUNT(*) FROM ExceptionLog;'''
df = pd.read_sql(query, conn)
print(df)

   count
0     20


/var/folders/44/s9hsvm1943z9xb042mqxxtgc0000gn/T/ipykernel_34180/3677073409.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [4]:
### DO NOT RUN THIS CELL MULTIPLE TIMES WITHOUT DELETING THE DATA OUT OF THE TABLE FIRST ###
### Creates the ReqType table -- should have 7 records when complete ###

df = pd.read_csv('reqtype.csv')

for x in df.index:
    
    query = f"""INSERT INTO ReqType (rtid, typename, typedesc)
            VALUES ({df['rtid'].loc[x]}, '{df['typename'].loc[x]}', '{df['typedesc'].loc[x]}')"""
    
    cursor.execute(query)
    
    conn.commit()
    
query = '''SELECT * FROM ReqType;'''
df = pd.read_sql(query, conn)
print(df)

   rtid      typename                                           typedesc
0     1  connectivity       Issues related to cable, phone, and internet
1     2         doors  Issues related to the function of any door fou...
2     3    electrical    Electrical issues (lights, outlets, appliances)
3     5          hvac  Issues related to heating, ventilation, and/or...
4     6      plumbing              Plumbing issues found within the unit
5     8       windows       Issues related to any window within the unit
6     9         other  Other issues not readily described by any of t...


/var/folders/44/s9hsvm1943z9xb042mqxxtgc0000gn/T/ipykernel_34180/3963926775.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [5]:
### DO NOT RUN THIS CELL MULTIPLE TIMES WITHOUT DELETING THE DATA OUT OF THE TABLE FIRST ###
### Creates the ServReq table -- should have 114 records when complete ###

df = pd.read_csv('servreq.csv')

for x in df.index:
    
    query = f"""INSERT INTO ServReq (tenantid, rtid, bldgid, reqdate, unit, reqdescription)
            VALUES ({df['tenantid'].loc[x]}, {df['rtid'].loc[x]}, {df['bldgid'].loc[x]}, 
            '{df['reqdate'].loc[x]}', {df['unit'].loc[x]}, '{df['reqdescription'].loc[x]}')"""
    
    cursor.execute(query)
    
    conn.commit()
    
query = '''SELECT COUNT(*) FROM ServReq;'''
df = pd.read_sql(query, conn)
print(df)

/var/folders/44/s9hsvm1943z9xb042mqxxtgc0000gn/T/ipykernel_34180/4148650442.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   count
0    114


In [6]:
### DO NOT RUN THIS CELL MULTIPLE TIMES WITHOUT DELETING THE DATA OUT OF THE TABLE FIRST ###
### Creates the Tenant table -- should have 700 records when complete ###
### THIS ONE MIGHT TAKE A COUPLE MINUTES ###

df = pd.read_csv('tenant.csv')

for x in df.index:
    
    query = f"""INSERT INTO tenant (fname, lname, phonenum)
            VALUES ('{df['fname'].loc[x]}', '{df['lname'].loc[x]}', '{df['phone'].loc[x]}')"""
    
    cursor.execute(query)
    
    conn.commit()
    
query = '''SELECT COUNT(*) FROM Tenant;'''
df = pd.read_sql(query, conn)
print(df)

   count
0    700


/var/folders/44/s9hsvm1943z9xb042mqxxtgc0000gn/T/ipykernel_34180/739666211.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
